Pruebas

In [13]:
import numpy as np
import scipy
import scipy.io
from procesar_tableros import *
from filtrar_datos import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import xlsxwriter
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

class RBFNetwork:
    def __init__(self, num_centers, sigma):
        self.num_centers = num_centers
        self.sigma = sigma
        self.centers = None
        self.weights = None

    def _calculate_activations(self, X):
        distances = scipy.spatial.distance.cdist(X, self.centers)
        activations = np.exp(-distances**2 / (2 * self.sigma**2))
        return activations

    def fit(self, X, y):
        print("Ajustando la red RBF...")
        indices = np.random.choice(X.shape[0], self.num_centers, replace=False)
        self.centers = X[indices]
        activations = self._calculate_activations(X)
        # Resolviendo el sistema de ecuaciones utilizando descomposición QR
        Q, R = np.linalg.qr(activations)
        self.weights = np.linalg.solve(R, Q.T @ y)
        print("Ajuste completado.")

    def predict(self, X):
        activations = self._calculate_activations(X)
        return activations @ self.weights

# Cargar datos
print("Cargando datos...")
file_path_1bit = "./10x10/10x10/1bit/DATASET_v7.mat"
file_path_2bit = "./10x10/10x10/2bit/DATASET_v7.mat"

mat_data1b = scipy.io.loadmat(file_path_1bit)
mat_data2b = scipy.io.loadmat(file_path_2bit)
print("Datos cargados.")

# Obtenemos los dataset
dataset1b = mat_data1b['DATASET']
dataset2b = mat_data2b['DATASET']

# Acceder a las variables dentro del archivo (1bit)
labels_1b = dataset1b['labels']
inputs_1b = dataset1b['input']
bits_1b = dataset1b['bits']
beams_1b = dataset1b['beams']

# Acceder a las variables dentro del archivo (2bit)
labels_2b = dataset2b['labels']
inputs_2b = dataset2b['input']
bits_2b = dataset2b['bits']
beams_2b = dataset2b['beams']

# Filtrar datos de 1 bit
print("Filtrando datos de 1 bit...")
datos1b_filtrados_input, datos1b_filtrados_labels = filtrar_datos_en_rango(inputs_1b, labels_1b)

# Filtrar datos de 2 bits
print("Filtrando datos de 2 bits...")
datos2b_filtrados_input, datos2b_filtrados_labels = filtrar_datos_en_rango(inputs_2b, labels_2b)

print("Datos filtrados listos.")

# Procesar los tableros
print("Procesando tableros...")

# 1 bit
tableros_procesados_1b = procesar_tableros(datos1b_filtrados_labels, 1)

# 2 bit
tableros_procesados_2b = procesar_tableros(datos2b_filtrados_labels, 2)

# Reshape tableros
rs_tableros_procesados_1b = np.transpose(tableros_procesados_1b, axes=(1,2,0))
rs_tableros_procesados_2b = np.transpose(tableros_procesados_2b, axes=(1,2,0))

# Comprobar dimensiones
print("Comprobando dimensiones...")
print("Dimensiones de datos1b_filtrados_labels:", datos1b_filtrados_labels.shape)
print("Dimensiones de rs_tableros_procesados_1b:", rs_tableros_procesados_1b.shape)

if datos1b_filtrados_labels.shape == rs_tableros_procesados_1b.shape:
    print("Las dimensiones coinciden. Usando rs_tableros_procesados_1b.")
    labels_to_use = rs_tableros_procesados_1b
else:
    print("Las dimensiones no coinciden. Usando datos1b_filtrados_labels.")
    labels_to_use = datos1b_filtrados_labels

# Dividir los datos en conjuntos de entrenamiento y prueba para la celda (0,0)
print("Dividiendo datos en conjuntos de entrenamiento y prueba...")
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    datos1b_filtrados_input, labels_to_use[0, 0, :].reshape(-1, 1), test_size=0.4, random_state=42
)

print("Datos divididos.")

# Definir valores de alpha y sigma para las pruebas
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
sigmas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Inicializar matrices para almacenar resultados
results_mse = np.zeros((len(alphas), len(sigmas)))
results_mae = np.zeros((len(alphas), len(sigmas)))
results_r2 = np.zeros((len(alphas), len(sigmas)))

# Realizar pruebas variando num_centers y sigma
for i, alpha in enumerate(alphas):
    num_centers = int(np.ceil(len(X_train1) * alpha))
    for j, sigma in enumerate(sigmas):
        print(f"Probando con num_centers = {num_centers} y sigma = {sigma}...")
        rbf_net = RBFNetwork(num_centers=num_centers, sigma=sigma)
        rbf_net.fit(X_train1, y_train1)
        predictions_test = rbf_net.predict(X_test1)
        mse_test = mean_squared_error(y_test1, predictions_test)
        mae_test = mean_absolute_error(y_test1, predictions_test)
        r2_test = r2_score(y_test1, predictions_test)
        results_mse[i, j] = mse_test
        results_mae[i, j] = mae_test
        results_r2[i, j] = r2_test
        print(f"MSE en conjunto de prueba con num_centers = {num_centers} y sigma = {sigma}: {mse_test}")
        print(f"MAE en conjunto de prueba con num_centers = {num_centers} y sigma = {sigma}: {mae_test}")
        print(f"R2 en conjunto de prueba con num_centers = {num_centers} y sigma = {sigma}: {r2_test}")

# Guardar resultados en un archivo Excel
print("Guardando resultados en un archivo Excel...")
workbook = xlsxwriter.Workbook('resultados_rbf.xlsx')
worksheet_mse = workbook.add_worksheet('MSE')
worksheet_mae = workbook.add_worksheet('MAE')
worksheet_r2 = workbook.add_worksheet('R2')

worksheets = {'MSE': worksheet_mse, 'MAE': worksheet_mae, 'R2': worksheet_r2}
results = {'MSE': results_mse, 'MAE': results_mae, 'R2': results_r2}

for key, worksheet in worksheets.items():
    worksheet.write(0, 0, 'num_centers \\ sigma')
    for j, sigma in enumerate(sigmas):
        worksheet.write(0, j + 1, sigma)
    for i, alpha in enumerate(alphas):
        num_centers = int(np.ceil(len(X_train1) * alpha))
        worksheet.write(i + 1, 0, num_centers)
        for j, sigma in enumerate(sigmas):
            worksheet.write(i + 1, j + 1, results[key][i, j])

workbook.close()
print("Resultados guardados en 'resultados_rbf.xlsx'.")

# Graficar resultados en 3D
print("Generando gráfica 3D de resultados...")
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(sigmas, [int(np.ceil(len(X_train1) * alpha)) for alpha in alphas])

for key, Z in results.items():
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis')
    ax.set_xlabel('Sigma')
    ax.set_ylabel('Num Centers')
    ax.set_zlabel(key)
    plt.title(f'Superficie de {key} en función de Sigma y Num Centers')
    plt.show()

# Graficar resultados en 2D
print("Generando gráfica 2D de resultados...")
for key, Z in results.items():
    plt.figure(figsize=(10, 6))
    for j, sigma in enumerate(sigmas):
        plt.plot([int(np.ceil(len(X_train1) * alpha)) for alpha in alphas], Z[:, j], marker='o', label=f'Sigma {sigma}')
    
    plt.xlabel('Num Centers')
    plt.ylabel(key)
    plt.legend()
    plt.title(f'{key} en función de Num Centers para diferentes valores de Sigma')
    plt.grid(True)
    plt.show()


Cargando datos...
Datos cargados.
Filtrando datos de 1 bit...
Filtrando datos de 2 bits...
Datos filtrados listos.
Procesando tableros...
Comprobando dimensiones...
Dimensiones de datos1b_filtrados_labels: (10, 10, 9180)
Dimensiones de rs_tableros_procesados_1b: (10, 10, 9180)
Las dimensiones coinciden. Usando rs_tableros_procesados_1b.
Dividiendo datos en conjuntos de entrenamiento y prueba...
Datos divididos.
Probando con num_centers = 551 y sigma = 0.2...
Ajustando la red RBF...
Ajuste completado.
MSE en conjunto de prueba con num_centers = 551 y sigma = 0.2: 0.01715685665582201
MAE en conjunto de prueba con num_centers = 551 y sigma = 0.2: 0.017156877968422385
R2 en conjunto de prueba con num_centers = 551 y sigma = 0.2: -0.017455978753962542
Probando con num_centers = 551 y sigma = 0.3...
Ajustando la red RBF...
Ajuste completado.
MSE en conjunto de prueba con num_centers = 551 y sigma = 0.3: 0.0171505986348648
MAE en conjunto de prueba con num_centers = 551 y sigma = 0.3: 0.01717

LinAlgError: SVD did not converge

Guardar resultados

In [ ]:
print("Pruebas completadas.")

# Definir valores de alpha y sigma para las pruebas
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
sigmas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Inicializar matrices para almacenar resultados
results_mse = np.zeros((len(alphas), len(sigmas)))
results_mae = np.zeros((len(alphas), len(sigmas)))
results_r2 = np.zeros((len(alphas), len(sigmas)))

# Realizar pruebas variando num_centers y sigma
for i, alpha in enumerate(alphas):
    num_centers = int(np.ceil(len(X_train1) * alpha))
    for j, sigma in enumerate(sigmas):
        print(f"Probando con num_centers = {num_centers} y sigma = {sigma}...")
        rbf_net = RBFNetwork(num_centers=num_centers, sigma=sigma)
        rbf_net.fit(X_train1, y_train1)
        predictions_test = rbf_net.predict(X_test1)
        mse_test = mean_squared_error(y_test1, predictions_test)
        mae_test = mean_absolute_error(y_test1, predictions_test)
        r2_test = r2_score(y_test1, predictions_test)
        results_mse[i, j] = mse_test
        results_mae[i, j] = mae_test
        results_r2[i, j] = r2_test
        print(f"MSE en conjunto de prueba con num_centers = {num_centers} y sigma = {sigma}: {mse_test}")
        print(f"MAE en conjunto de prueba con num_centers = {num_centers} y sigma = {sigma}: {mae_test}")
        print(f"R2 en conjunto de prueba con num_centers = {num_centers} y sigma = {sigma}: {r2_test}")

# Guardar resultados en un archivo Excel
print("Guardando resultados en un archivo Excel...")
workbook = xlsxwriter.Workbook('resultados_rbf.xlsx')
worksheet_mse = workbook.add_worksheet('MSE')
worksheet_mae = workbook.add_worksheet('MAE')
worksheet_r2 = workbook.add_worksheet('R2')

worksheets = {'MSE': worksheet_mse, 'MAE': worksheet_mae, 'R2': worksheet_r2}
results = {'MSE': results_mse, 'MAE': results_mae, 'R2': results_r2}

for key, worksheet in worksheets.items():
    worksheet.write(0, 0, 'num_centers \\ sigma')
    for j, sigma in enumerate(sigmas):
        worksheet.write(0, j + 1, sigma)
    for i, alpha in enumerate(alphas):
        num_centers = int(np.ceil(len(X_train1) * alpha))
        worksheet.write(i + 1, 0, num_centers)
        for j, sigma in enumerate(sigmas):
            worksheet.write(i + 1, j + 1, results[key][i, j])

workbook.close()
print("Resultados guardados en 'resultados_rbf.xlsx'.")

# Graficar resultados en 3D
print("Generando gráfica 3D de resultados...")
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(sigmas, [int(np.ceil(len(X_train1) * alpha)) for alpha in alphas])

for key, Z in results.items():
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis')
    ax.set_xlabel('Sigma')
    ax.set_ylabel('Num Centers')
    ax.set_zlabel(key)
    plt.title(f'Superficie de {key} en función de Sigma y Num Centers')
    plt.show()

# Graficar resultados en 2D
print("Generando gráfica 2D de resultados...")
for key, Z in results.items():
    plt.figure(figsize=(10, 6))
    for j, sigma in enumerate(sigmas):
        plt.plot([int(np.ceil(len(X_train1) * alpha)) for alpha in alphas], Z[:, j], marker='o', label=f'Sigma {sigma}')
    
    plt.xlabel('Num Centers')
    plt.ylabel(key)
    plt.legend()
    plt.title(f'{key} en función de Num Centers para diferentes valores de Sigma')
    plt.grid(True)
    plt.show()



Pruebas completadas.
Probando con num_centers = 551 y sigma = 0.2...


NameError: name 'reg_lambda' is not defined